# How to select examples from a LangSmith dataset

```{=mdx}

:::tip Prerequisites

- [Chat models](/docs/concepts/chat_models)
- [Few-shot-prompting](/docs/concepts/few_shot_prompting)
- [LangSmith](/docs/concepts/#langsmith)

:::


:::note Compatibility

- `langsmith` >= 0.1.43

:::

```

LangSmith datasets have built-in support for similarity search, making them a great tool for building and querying few-shot examples.

In this guide we'll see how to use an indexed LangSmith dataset as a few-shot example selector.

## Setup

Before getting started make sure you've [created a LangSmith account](https://smith.langchain.com/) and set your credentials:

```typescript
process.env.LANGSMITH_API_KEY="your-api-key"
process.env.LANGSMITH_TRACING="true"
```

We'll need to install the `langsmith` SDK. In this example we'll also make use of `langchain` and `@langchain/anthropic`:

```{=mdx}

import Npm2Yarn from "@theme/Npm2Yarn"

<Npm2Yarn>
  langsmith langchain @langchain/anthropic @langchain/core zod zod-to-json-schema
</Npm2Yarn>

```

Now we'll clone a public dataset and turn on indexing for the dataset. We can also turn on indexing via the [LangSmith UI](https://docs.smith.langchain.com/how_to_guides/datasets/index_datasets_for_dynamic_few_shot_example_selection).

We'll create a clone the [Multiverse math few shot example dataset](https://blog.langchain.dev/few-shot-prompting-to-improve-tool-calling-performance/).

This enables searching over the dataset, and will make sure that anytime we update/add examples they are also indexed.

The first step to creating a clone is to read the JSON file containing the examples and convert them to the format expected by LangSmith for creating examples:

In [1]:
import { Client as LangSmithClient } from 'langsmith';
import { z } from 'zod';
import { zodToJsonSchema } from 'zod-to-json-schema';
import fs from "fs/promises";

// Read the example dataset and convert to the format expected by the LangSmith API
// for creating new examples
const examplesJson = JSON.parse(
  await fs.readFile("../../data/ls_few_shot_example_dataset.json", "utf-8")
);

let inputs: Record<string, any>[] = [];
let outputs: Record<string, any>[] = [];
let metadata: Record<string, any>[] = [];

examplesJson.forEach((ex) => {
  inputs.push(ex.inputs);
  outputs.push(ex.outputs);
  metadata.push(ex.metadata);
});

// Define our input schema as this is required for indexing
const inputsSchema = zodToJsonSchema(z.object({
  input: z.string(),
  system: z.boolean().optional(),
}));

const lsClient = new LangSmithClient();

await lsClient.deleteDataset({ datasetName: "multiverse-math-examples-for-few-shot-example" })

const dataset = await lsClient.createDataset("multiverse-math-examples-for-few-shot-example", {
  inputsSchema,
});

const createdExamples = await lsClient.createExamples({
  inputs,
  outputs,
  metadata,
  datasetId: dataset.id,
})


In [2]:
await lsClient.indexDataset({ datasetId: dataset.id });

Once the dataset is indexed, we can search for similar examples like so:

In [6]:
const examples = await lsClient.similarExamples(
  { input: "whats the negation of the negation of the negation of 3" },
  dataset.id,
  3,
)
console.log(examples.length)

3


In [7]:
console.log(examples[0].inputs.input)

evaluate the negation of -100


For this dataset the outputs are an entire chat history:

In [8]:
console.log(examples[1].outputs.output)

[
  {
    id: 'cbe7ed83-86e1-4e46-89de-6646f8b55cef',
    type: 'system',
    content: 'You are requested to solve math questions in an alternate mathematical universe. The operations have been altered to yield different results than expected. Do not guess the answer or rely on your  innate knowledge of math. Use the provided tools to answer the question. While associativity and commutativity apply, distributivity does not. Answer the question using the fewest possible tools. Only include the numeric response without any clarifications.',
    additional_kwargs: {},
    response_metadata: {}
  },
  {
    id: '04946246-09a8-4465-be95-037efd7dae55',
    type: 'human',
    content: 'if one gazoink is 4 badoinks, each of which is 6 foos, each of wich is 3 bars - how many bars in 3 gazoinks?',
    example: false,
    additional_kwargs: {},
    response_metadata: {}
  },
  {
    id: 'run-d6f0954e-b21b-4ea8-ad98-0ee64cfc824e-0',
    type: 'ai',
    content: [ [Object] ],
    example: false,
  

The search returns the examples whose inputs are most similar to the query input. We can use this for few-shot prompting a model. The first step is to create a series of math tools we want to allow the model to call:

In [9]:
import { tool } from '@langchain/core/tools';
import { z } from 'zod';

const add = tool((input) => {
  return (input.a + input.b).toString();
}, {
  name: "add",
  description: "Add two numbers",
  schema: z.object({
    a: z.number().describe("The first number to add"),
    b: z.number().describe("The second number to add"),
  }),
});

const cos = tool((input) => {
  return Math.cos(input.angle).toString();
}, {
  name: "cos",
  description: "Calculate the cosine of an angle (in radians)",
  schema: z.object({
    angle: z.number().describe("The angle in radians"),
  }),
});

const divide = tool((input) => {
  return (input.a / input.b).toString();
}, {
  name: "divide",
  description: "Divide two numbers",
  schema: z.object({
    a: z.number().describe("The dividend"),
    b: z.number().describe("The divisor"),
  }),
});

const log = tool((input) => {
  return Math.log(input.value).toString();
}, {
  name: "log",
  description: "Calculate the natural logarithm of a number",
  schema: z.object({
    value: z.number().describe("The number to calculate the logarithm of"),
  }),
});

const multiply = tool((input) => {
  return (input.a * input.b).toString();
}, {
  name: "multiply",
  description: "Multiply two numbers",
  schema: z.object({
    a: z.number().describe("The first number to multiply"),
    b: z.number().describe("The second number to multiply"),
  }),
});

const negate = tool((input) => {
  return (-input.a).toString();
}, {
  name: "negate",
  description: "Negate a number",
  schema: z.object({
    a: z.number().describe("The number to negate"),
  }),
});

const pi = tool(() => {
  return Math.PI.toString();
}, {
  name: "pi",
  description: "Return the value of pi",
  schema: z.object({}),
});

const power = tool((input) => {
  return Math.pow(input.base, input.exponent).toString();
}, {
  name: "power",
  description: "Raise a number to a power",
  schema: z.object({
    base: z.number().describe("The base number"),
    exponent: z.number().describe("The exponent"),
  }),
});

const sin = tool((input) => {
  return Math.sin(input.angle).toString();
}, {
  name: "sin",
  description: "Calculate the sine of an angle (in radians)",
  schema: z.object({
    angle: z.number().describe("The angle in radians"),
  }),
});

const subtract = tool((input) => {
  return (input.a - input.b).toString();
}, {
  name: "subtract",
  description: "Subtract two numbers",
  schema: z.object({
    a: z.number().describe("The number to subtract from"),
    b: z.number().describe("The number to subtract"),
  }),
});

In [17]:
import { ChatOpenAI } from "@langchain/openai";
import { HumanMessage, SystemMessage, BaseMessage, BaseMessageLike } from "@langchain/core/messages";
import { RunnableLambda } from "@langchain/core/runnables";
import { Client as LangSmithClient, Example } from "langsmith";
import { coerceMessageLikeToMessage } from "@langchain/core/messages";

const client = new LangSmithClient();

async function similarExamples(input: Record<string, any>): Promise<Record<string, any>> {
  const examples = await client.similarExamples(input, dataset.id, 5);
  return { ...input, examples };
}

function constructPrompt(input: { examples: Example[], input: string }): BaseMessage[] {
  const instructions = "You are great at using mathematical tools.";
  let messages: BaseMessage[] = []
  
  for (const ex of input.examples) {
    // Assuming ex.outputs.output is an array of message-like objects
    messages = messages.concat(ex.outputs.output.flatMap((msg: BaseMessageLike) => coerceMessageLikeToMessage(msg)));
  }
  
  const examples = messages.filter(msg => msg._getType() !== 'system');
  examples.forEach((ex) => {
    if (ex._getType() === 'human') {
      ex.name = "example_user";
    } else {
      ex.name = "example_assistant";
    }
  });

  return [new SystemMessage(instructions), ...examples, new HumanMessage(input.input)];
}

const llm = new ChatOpenAI({
  model: "gpt-4o",
  temperature: 0,
});
const tools = [add, cos, divide, log, multiply, negate, pi, power, sin, subtract];
const llmWithTools = llm.bindTools(tools);

const exampleSelector = new RunnableLambda(
  { func: similarExamples }
).withConfig({ runName: "similarExamples" });

const chain = exampleSelector.pipe(
  new RunnableLambda({
    func: constructPrompt
  }).withConfig({
    runName: "constructPrompt"
  })
).pipe(llmWithTools);

In [16]:
const aiMsg = await chain.invoke({ input: "whats the negation of the negation of 3", system: false })
console.log(aiMsg.tool_calls)

[
  {
    name: 'negate',
    args: { a: 3 },
    type: 'tool_call',
    id: 'call_SX0dmb4AbFu39KkGQDqPXQwa'
  }
]


Looking at the LangSmith trace, we can see that relevant examples were pulled in in the `similarExamples` step and passed as messages to ChatOpenAI: https://smith.langchain.com/public/20e09618-0746-4973-9382-5b36c3f27083/r.